In [73]:
import collections
import csv
import itertools
import matplotlib.pyplot as plt
import json
import os
import pathlib

import pandas as pd

import IPython.core.display as ipy_display


JSON_FILE = pathlib.Path("/Users/shinewu/tmp/repeat_rec.txt")

In [74]:
import snowflake.connector as sf_conn

_SNOWFLAKE_CONTEXT = None


def snowflake_run_query(sql):
    global _SNOWFLAKE_CONTEXT
    if _SNOWFLAKE_CONTEXT is None:
        _SNOWFLAKE_CONTEXT = sf_conn.connect(
            user="xiang_wu",
            password=os.getenv("SNOWFLAKE_PASSWORD"),
            account="opensea.us-east-1",
        )

    cs = _SNOWFLAKE_CONTEXT.cursor()
    cs.execute(sql)
    return cs.fetch_pandas_all()


def fetch_metadata_for_collections(collection_ids):
    concat_collection_ids = ", ".join(collection_ids)
    sql = f"""
        SELECT
          a.NAME,
          a.SLUG,
          a.DEFAULT_CHAIN_ID AS CHAIN,
          a.BLACKLISTED AS BLK,
          a.HIDDEN AS HID,
          a.IS_DELISTED AS DEL,
          a.IS_LISTABLE AS LIS,
          a.IS_NSFW AS NSFW,
          b.THIRTY_DAY_SALES,
          b.THIRTY_DAY_VOLUME * IFF(a.DEFAULT_CHAIN_ID = 1, 1800, 40) AS THIRTY_DAY_VOLUME,
          b.MARKET_CAP * IFF(a.DEFAULT_CHAIN_ID = 1, 1800, 40) AS MARKET_CAP,
          b.NUM_OWNERS,
          b.TOTAL_SUPPLY,         
          a.EXTERNAL_URL,
          a.DISCORD_URL,
          a.ID,
          a.IMAGE_URL
        FROM
          PC_FIVETRAN_DB.PUBLIC.API_COLLECTION a
          LEFT JOIN PC_FIVETRAN_DB.PUBLIC.API_COLLECTIONSTATS b
          ON a.ID = b.COLLECTION_ID
        WHERE
          a.ID in ({concat_collection_ids})
        LIMIT {len(collection_ids)}
    """
    return snowflake_run_query(sql)

In [75]:
def display_recommendation(input_collection, output_collections):
    uniq_collections = {input_collection} | set(output_collections)
    df = fetch_metadata_for_collections(list(uniq_collections))
    df["image_display"] = [
        f'<img src="{image_url}" alt="{image_url}">' for image_url in df.IMAGE_URL
    ]
    cols = df.columns
    assert cols[-1] == "image_display"
    new_cols = list(cols[:-1])
    new_cols.insert(3, cols[-1])
    df = df[new_cols]

    input_df = pd.DataFrame({"ID": [int(input_collection)]})
    input_df = pd.merge(input_df, df)
    ipy_display.display(ipy_display.HTML(input_df.to_html(escape=False)))

    output_df = pd.DataFrame({"ID": [int(x) for x in output_collections]})
    output_df = pd.merge(output_df, df)
    ipy_display.display(ipy_display.HTML(output_df.to_html(escape=False)))

In [76]:
NUM_ROWS = None

ITEM2ITEM = {}

with JSON_FILE.open() as f:
    for row in itertools.islice(f, NUM_ROWS):
        json_obj = json.loads(row)
        input_item = str(json_obj["input"]["itemId"])
        output_items = json_obj["output"]["recommendedItems"]
        ITEM2ITEM[input_item] = output_items

In [77]:
item2item = list(ITEM2ITEM.items())
item2item.sort()

In [81]:
PICK = 9
icol = item2item[PICK][0]
ocols = item2item[PICK][1][:25]
print(icol, ocols)
display_recommendation(icol, ocols)

6131487 ['264056', '4763446', '7750235', '10563', '5831065', '8268014', '8146804', '2252197', '8125810', '7112915', '8406538', '3150', '5056253', '389957', '5175201', '7809601', '6677756', '8076460', '6525063', '5954579', '5102617', '5187657', '7651570', '8239216', '4247568']


,ID,NAME,SLUG,CHAIN,image_display,BLK,HID,DEL,LIS,NSFW,THIRTY_DAY_SALES,THIRTY_DAY_VOLUME,MARKET_CAP,NUM_OWNERS,TOTAL_SUPPLY,EXTERNAL_URL,DISCORD_URL,IMAGE_URL
0,6131487,BW portraits,bw-portraits,3,,False,False,False,True,False,0.0,0.0,0.0,1.0,7.0,None,None,https://lh3.googleusercontent.com/K6mTHiuTPAHHEkLwFe5liqN5SApVdCy5ywE_7dEiaZU2eERC3B1HUHFtQwaikxZtAjcRX_Uqj9R8kniNd-RFPr3ymLjPaZG9sDO1Kw=s120


,ID,NAME,SLUG,CHAIN,image_display,BLK,HID,DEL,LIS,NSFW,THIRTY_DAY_SALES,THIRTY_DAY_VOLUME,MARKET_CAP,NUM_OWNERS,TOTAL_SUPPLY,EXTERNAL_URL,DISCORD_URL,IMAGE_URL
0,264056,Parallel Alpha,parallelalpha,1,,False,False,False,True,False,2595.0,7.487860e+05,1.995328e+05,45641.0,538.0,http://www.parallel.life,https://discord.gg/parallelalpha,https://lh3.googleusercontent.com/Nnp8Pdo6EidK7eBduGnAn_JBvFsYGhNGMJ_fHJ_mzGMN_2Khu5snL5zmiUMcSsIqtANh19KqxXDs0iNq_aYbKC5smO3hiCSw9PlL=s120
1,4763446,Azuki,azuki,1,,False,False,False,True,False,3662.0,8.698614e+07,2.238990e+08,5157.0,10000.0,http://www.azuki.com,https://discord.gg/azuki,https://lh3.googleusercontent.com/H8jOCJuQokNqGBpkBN5wk1oZwO7LM8bNnrHCaekV2nKjnCqw6UB5oaH8XyNeBDj6bA_n1mjejzhFQUP3O1NfjFLHr3FOaeHcTOOT=s120
2,7750235,tubby cats,tubby-cats,1,,False,False,False,True,False,1172.0,4.070051e+05,8.048682e+06,9061.0,20000.0,http://tubbycats.xyz,https://discord.gg/tubbycatsnft,https://lh3.googleusercontent.com/TyPJi06xkDXOWeK4wYBCIskRcSJpmtVfVcJbuxNXDVsC39IC_Ls5taMlxpZPYMoUtlPH7YkQ4my1nwUGDIB5C01r97TPlYhkolk-TA=s120
3,10563,Town Star,town-star,1,,False,False,False,True,False,6382.0,2.879339e+06,2.337858e+05,46279.0,536.0,https://www.gala.games/,None,https://lh3.googleusercontent.com/wp7V1K4Ezhi8yHU6LFpff-GMsjend65nRiwNz5pGAjCwk7ybznEpfUvXiIxQoOWBmeN00AGjVTRhLUqY4fee1fus=s120
4,5831065,MekaApes Game by OogaVerse,mekaapes,1,,False,False,False,True,False,322.0,2.077628e+04,1.719788e+06,2036.0,39506.0,http://mekaapes.game,https://discord.gg/oogaverse,https://lh3.googleusercontent.com/JyBj6Saua8aKejfxRGvndGs_qp-AwSmXMDG780QR6zlIocvNPUNm0fy8c5QVdCPBv4j0VcIng2JhVwR2P-H8cDSMyhpw2zCM-WOf-g=s120
5,8268014,Gutter Juice,gutter-juice,1,,False,False,False,True,False,1105.0,5.228613e+05,5.152745e+06,3804.0,12352.0,https://guttercatgang.com,https://discord.gg/guttercatgang,https://lh3.googleusercontent.com/xsApnCm19QrZDa8uPuZ8GBoNxRbKW5aPZQsUedWh3giAEM5Oo7dEDDjAQcEdVHDvbDlHfmOOv2mqyv5owaGHWvmDfc4zZ8Nq-gRxbw=s120
6,8146804,Loser Club Official,loser-club-official,1,,False,False,False,True,False,1168.0,5.474665e+05,4.189635e+06,5159.0,10000.0,https://loserclub.io/,https://discord.gg/loserclub,https://lh3.googleusercontent.com/lboeLdy4a1Bodfib2FRXkqLeWPCaGbCeWs6GdOMQhXqO7r41feS7iRkyb99qoHaxpmYb9NF7wnvyjt7VU5Jwo2YGrAst5Dgtm_RpRQ=s120
7,2252197,Metroverse Genesis,metroverse-genesis,1,,False,False,False,True,False,619.0,7.974552e+05,7.693172e+06,295.0,10000.0,https://metroverse.com,https://discord.gg/metroverse,https://lh3.googleusercontent.com/sHO92R4xynTQtuC-DZ1B5qC5ZJe3n3Y46iufDza_sDXHuSj1zhhguoj-YjKN0pxh3b0Lg0Rz8Cp_BQliEh1o_-puGOVDDH8H7bzqcQ=s120
8,8125810,DourDarcels,dourdarcels,1,,False,False,False,True,False,720.0,3.539171e+05,4.164527e+06,4838.0,10000.0,http://dourdarcels.io,https://discord.gg/dourdarcels,https://lh3.googleusercontent.com/dz1IK4dZ8rKEvxWftwUgyEo0mIUXBVMe6XYx12l6sqPJjYNwpvIPR5B80jqHQLiY1fvHXfaG3MKZFi5YzzUvfbxdlr4IT4vPC2WWrQ=s120
9,7112915,Gooniez Gang Official,gooniezgangofficial,1,,False,False,False,True,False,317.0,4.348255e+04,1.282791e+06,5434.0,8888.0,https://gooniezgang.com/,https://discord.gg/2aGWamceRt,https://lh3.googleusercontent.com/OI59gxLlnJR1Vl2KffEVx_dK6hwpnAv2fkypmPuAMgJTJCFTEMuY4m40H6gotvEJN544Zg-XbcH49JC7diyPwmLGKUV8K2Vyt2YQKg=s120
